In [44]:
import numpy as np
from mnist import MNIST

# Load the MNIST dataset
mndata = MNIST("C:\\Users\\jkobo\\Downloads\\mnist\\data")
mndata.gz = True
train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

# Convert to NumPy arrays for easier manipulation
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)



In [45]:
# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# One-hot encode the labels
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_labels = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        one_hot_labels[i][labels[i]] = 1
    return one_hot_labels

num_classes = 10  # 0 to 9
train_labels = one_hot_encode(train_labels, num_classes)
test_labels = one_hot_encode(test_labels, num_classes)

In [46]:
# Define the neural network architecture
input_size = 784  # 28x28 image dimensions
hidden1_size = 128
hidden2_size = 64
output_size = num_classes

# Initialize weights and biases
np.random.seed(0)

weights_input_hidden1 = np.random.randn(input_size, hidden1_size)
bias_hidden1 = np.zeros(hidden1_size)
weights_hidden1_hidden2 = np.random.randn(hidden1_size, hidden2_size)
bias_hidden2 = np.zeros(hidden2_size)
weights_hidden2_output = np.random.randn(hidden2_size, output_size)
bias_output = np.zeros(output_size)


In [47]:
# Activation function (sigmoid)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of the sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)


In [48]:
# Activation function (ReLU)
def relu(x):
    return np.maximum(0, x)

# Derivative of the ReLU function
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [49]:
# Define the learning rate
learning_rate = 0.0001

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    # Forward propagation
    hidden1_input = np.dot(train_images, weights_input_hidden1) + bias_hidden1
    hidden1_output = relu(hidden1_input)
    hidden2_input = np.dot(hidden1_output, weights_hidden1_hidden2) + bias_hidden2
    hidden2_output = relu(hidden2_input)
    output_input = np.dot(hidden2_output, weights_hidden2_output) + bias_output
    output = relu(output_input)

    # Calculate the loss (mean squared error)
    loss = np.mean((output - train_labels) ** 2)

    # Backpropagation
    d_output = 2 * (output - train_labels)
    d_output *= relu_derivative(output)
    d_hidden2 = np.dot(d_output, weights_hidden2_output.T)
    d_hidden2 *= relu_derivative(hidden2_output)
    d_hidden1 = np.dot(d_hidden2, weights_hidden1_hidden2.T)
    d_hidden1 *= relu_derivative(hidden1_output)

    # Update weights and biases
    weights_hidden2_output -= np.dot(hidden2_output.T, d_output) * learning_rate
    bias_output -= np.sum(d_output, axis=0) * learning_rate
    weights_hidden1_hidden2 -= np.dot(hidden1_output.T, d_hidden2) * learning_rate
    bias_hidden2 -= np.sum(d_hidden2, axis=0) * learning_rate
    weights_input_hidden1 -= np.dot(train_images.T, d_hidden1) * learning_rate
    bias_hidden1 -= np.sum(d_hidden1, axis=0) * learning_rate

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}")


Epoch 1/10, Loss: 90807.2841
Epoch 2/10, Loss: 0.1000
Epoch 3/10, Loss: 0.1000
Epoch 4/10, Loss: 0.1000
Epoch 5/10, Loss: 0.1000
Epoch 6/10, Loss: 0.1000
Epoch 7/10, Loss: 0.1000
Epoch 8/10, Loss: 0.1000
Epoch 9/10, Loss: 0.1000
Epoch 10/10, Loss: 0.1000


In [50]:
# Testing the Neural Network
hidden1_input = np.dot(test_images, weights_input_hidden1) + bias_hidden1
hidden1_output = relu(hidden1_input)
hidden2_input = np.dot(hidden1_output, weights_hidden1_hidden2) + bias_hidden2
hidden2_output = relu(hidden2_input)
output_input = np.dot(hidden2_output, weights_hidden2_output) + bias_output
output = relu(output_input)

predicted_labels = np.argmax(output, axis=1)

accuracy = np.mean(predicted_labels == np.argmax(test_labels, axis=1))
print(f"Test accuracy: {accuracy * 100:.2f}%")

Test accuracy: 9.80%


powyższe nie działa - chyba można to wywalić  
poniższe działa

In [5]:
import numpy as np
from mnist import MNIST

# Load the MNIST dataset
mndata = MNIST("C:\\Users\\jkobo\\Downloads\\mnist\\data")  # Replace with the path to your MNIST data
mndata.gz = True

# Load the training and testing data
train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

# Convert to NumPy arrays for easier manipulation
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

# One-hot encode the labels
num_classes = 10
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

# Define the neural network architecture
input_size = 784  # 28x28 image dimensions
hidden_size = 128
output_size = num_classes

# Initialize weights and biases with Xavier initialization
np.random.seed(0)
weights_input_hidden = np.random.normal(0, np.sqrt(2 / (input_size + hidden_size)), size=(input_size, hidden_size))
bias_hidden = np.zeros(hidden_size)
weights_hidden_output = np.random.normal(0, np.sqrt(2 / (hidden_size + output_size)), size=(hidden_size, output_size))
bias_output = np.zeros(output_size)

# Softmax activation function for the output layer
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()


# Learning rate
learning_rate = 0.01

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    for i in range(len(train_images)):
        # Forward propagation
        hidden_input = np.dot(train_images[i], weights_input_hidden) + bias_hidden
        hidden_output = 1 / (1 + np.exp(-hidden_input))  # Sigmoid activation for the hidden layer
        output_input = np.dot(hidden_output, weights_hidden_output) + bias_output
        output = softmax(output_input)

        # Calculate the cross-entropy loss
        loss = -np.sum(train_labels[i] * np.log(output))

        # Backpropagation
        d_output = output - train_labels[i]
        d_hidden = np.dot(d_output, weights_hidden_output.T) * hidden_output * (1 - hidden_output)

        # Update weights and biases
        weights_hidden_output -= np.outer(hidden_output, d_output) * learning_rate
        bias_output -= d_output * learning_rate
        weights_input_hidden -= np.outer(train_images[i], d_hidden) * learning_rate
        bias_hidden -= d_hidden * learning_rate

    # Calculate the test accuracy
    correct = 0
    for i in range(len(test_images)):
        hidden_input = np.dot(test_images[i], weights_input_hidden) + bias_hidden
        hidden_output = 1 / (1 + np.exp(-hidden_input))
        output_input = np.dot(hidden_output, weights_hidden_output) + bias_output
        output = softmax(output_input)
        if np.argmax(output) == np.argmax(test_labels[i]):
            correct += 1

    test_accuracy = correct / len(test_images)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10, Loss: 0.1539, Test Accuracy: 92.51%
Epoch 2/10, Loss: 0.0720, Test Accuracy: 94.51%
Epoch 3/10, Loss: 0.0377, Test Accuracy: 95.85%
Epoch 4/10, Loss: 0.0228, Test Accuracy: 96.43%
Epoch 5/10, Loss: 0.0157, Test Accuracy: 96.76%
Epoch 6/10, Loss: 0.0113, Test Accuracy: 96.99%
Epoch 7/10, Loss: 0.0082, Test Accuracy: 97.19%
Epoch 8/10, Loss: 0.0060, Test Accuracy: 97.20%
Epoch 9/10, Loss: 0.0043, Test Accuracy: 97.27%
Epoch 10/10, Loss: 0.0032, Test Accuracy: 97.32%
